# RUN SCC-DFTB+CCS+PiNN Simulations using ASE 
Input: SKF/, CCS/ and PiNN/ folders from ParAutomatik

Simualtion type: 
- Geometry optimization
- MD Simulations 
- and more 

In [ ]:
PARAUTOMATIK_PATH='WRITE-Your-PATH-PARAUTOMATIK-Here'   # Add your installation path here

In [ ]:
import os 
import sys
from ase import Atoms
from ase import io
from ase.optimize import FIRE
from ase.io import read, write
from ase.calculators.dftb import Dftb
from ase.calculators.mixing import LinearCombinationCalculator
import json
from ccs.ase_tools.ccs_ase_calculator import CCS  
import numpy as np
import pinn 
sys.path.append(PARAUTOMATIK_PATH+'/pgm/CCS/')

base_dir=os.getcwd()
os.chdir(base_dir)

if not os.path.isdir("RUN_SIM/"):
    os.mkdir("RUN_SIM/")

os.chdir("RUN_SIM/")
cwd = os.getcwd() 
print('You are here',cwd) 

def setup_model(elements,spin,charge,**kwargs):

    # set up model:
    # 1: DFTB 
    os.environ["DFTB_PREFIX"] = base_dir+"/SKF/"
    DFTB_calc=Dftb(Hamiltonian_SCC='Yes',Hamiltonian_ShellResolvedSCC = 'Yes',Hamiltonian_SCCTolerance = '1.0E-006',Hamiltonian_Mixer= 'Anderson{',Hamiltonian_Mixer_MixingParameter='0.03',Hamiltonian_MaxSCCIterations = '1500',Hamiltonian_Filling = 'MethfesselPaxton{',Hamiltonian_Filling_Temperature = '0.001583407672623195',Hamiltonian_KPointsAndWeights = 'SupercellFolding {1 0 0 0 1 0 0 0 1 0.0 0.0 0.0}') 
    if spin>0:                  
        DFTB_calc.set(Hamiltonian_SpinPolarisation = 'Colinear{') 
        DFTB_calc.set(Hamiltonian_SpinPolarisation_UnpairedElectrons=str(spin))
        DFTB_calc.set(Hamiltonian_SpinConstants_ ='')
        for i in elements: 
            print(i)
            spinparam=get_spindata(i)
            string=f'Hamiltonian_SpinConstants_{i}'  
            print(string)
            DFTB_calc.set(**{string:spinparam})  
    if charge>0: 
        DFTB_calc.set(Hamiltonian_Charge =str(charge))
    print(DFTB_calc.todict())     

                     
    #2: CCS 
    with open(base_dir+'/CCS/CCS_params.json', 'r') as f:
        CCS_params = json.load(f)

    CCS_calc=CCS(CCS_params=CCS_params) 
    #3: PiNN 
    PiNN_calc = pinn.get_calc(base_dir+'/PiNN/') 
    

    # and the calculater becomes 
    calcs =[DFTB_calc,CCS_calc, PiNN_calc]
    weights=[1,1,1]
    calc=LinearCombinationCalculator(calcs, weights) 
    return calc            

def get_spindata(element): 
    if element=='Li':
        spinparam='{-0.019  -0.016  -0.016 -0.027}'
    if element=='C':
        spinparam='{-0.031 -0.025 -0.025 -0.023}'
    if element=='O':
        spinparam='{-0.035 -0.030 -0.030 -0.028}'
    if element=='H':
        spinparam='{-0.072}'
    return spinparam        

In [ ]:
# Geometry optimization
# import structure:
slab = io.read('../DFT_DB.db@0')
elements=['C','H','O'] 
spin=0
charge=0
slab.calc=setup_model(elements, spin, charge)
print(slab.calc)
relax = FIRE(slab, trajectory='opt.traj')
relax.run(fmax=0.05) 

In [ ]:
!ase gui out.traj

In [ ]:
# Read geometry optimized structure and run NVE Molecular dynamics 
from ase import units
from ase.io import read, write
from ase.io.trajectory import Trajectory
from ase.md import MDLogger
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.nptberendsen import NPTBerendsen
from ase.md.nvtberendsen import NVTBerendsen
from ase.md.verlet import VelocityVerlet

rng = np.random.default_rng(10)
atoms = read("opt.traj")
atoms.set_calculator(calc)
MaxwellBoltzmannDistribution(atoms, 873*units.kB, rng=rng)
dt = 0.5 * units.fs
steps = int(10)
dyn = VelocityVerlet(atoms, timestep=dt)
interval = int(1)
dyn.attach(MDLogger(dyn, atoms, 'output.log', mode="a"), interval=interval)
dyn.attach(Trajectory('output.traj', 'a', atoms).write, interval=interval)
dyn.run(steps)

In [ ]:
!ase gui output.traj 